## Some important notes: 

Here is the resubmission of this challenge. Thank you for giving me a second attempt. I modify the bitmap usage to run length encoding. In my initial submission, I mistakenly thought bitmap is the same as run length encoding. I did not have a background in compression, and I did the research when doing the quesiton. So I made a mistake in the first submission. In this submission, I have used run length encoding. 

This challenge is more about data processing, and machine learning techniques. The prepared data could be used to train machine learning or AI models to determine whether a parasite has cancer based on some "factors/predictors". I had a strong background in Machine learning and AI, a few years ago. When I had my undergraduate degree, in statiscis and comoputer science, my focus was on AI and machine learning. After the graduation and my master degree, I shifted my focus more to web development, working with cloud, frontend technologies and backend technologies, such as react, angular, node, python, fastapi, express, java spring boot, docker, microservice, etc. So I am no longer proficient in working with data processing and train models. But I did this question as best as I could. 

Therefore, this kind of challenge is no longer my strength at the moment of this challenge. But I am willing to do some review and refresh my memory if I could be considered as candidates for this type of role. Thank you

### Question 1:
Image by microscope: One representation is that I can use a bitmap with run length encoding. I initially intended to represent images as 2-d array in python, and I can utilize the GPU's parallel processing in pytorch for further analysis easily. But I notice that the question's requirement is to use as less storage/memory as possible. Therefore, a bitmap with run length encoding would more closely align with the requrements. 

Since the large amount of consecutive bits are of the same color(white or black), we will use 33 bits to store each run. For example, the 32 bits stores the length of the run, and the last bit stores the color (0 for white, 1 for black).

In the worst case, there are 10 billion runs(color alternates). Then there would be 1000000000 * 33 bits. This would be worse than using two d array. But only by a constant factor which is 33. In big O notation, the storage is O(n), suppose n is the number of bits in two d array, which is linear in the worse case for both 2d array representation, and bitmap representation. To translate to bytes, you will simply divde that number by 4

But in practice, this representation would be better. Since the situation is likely that in a single row of the 2d array representation, there are only constant number alternations of colors(because the blog is grouped together), then the storage becomes
O(log(n)), which would be better than the two d array

I would represent dye image in the same way, because given the situation of this challenge, the dye image is also binary(whether the dye is lit or not). Bitmap is great for binary representations. 

I also reserached if there are intensities of the dye involved, we will use other representations such as tree

### Qestion 2

In [ ]:
import cv2
import numpy as np
from tqdm import tqdm

In [ ]:
def rle_encode_large(binary_array):
    """
    Perform run-length encoding on a large 1D binary numpy array.
    This function processes the array in chunks to save memory.
    """
    chunk_size = 10_000_000  # Process 10 million elements at a time
    encoded_chunks = []
    
    for i in tqdm(range(0, len(binary_array), chunk_size), desc="Encoding"):
        chunk = binary_array[i:i+chunk_size]
        diff = np.diff(chunk)
        run_starts = np.where(diff != 0)[0] + 1
        run_starts = np.concatenate(([0], run_starts, [len(chunk)]))
        run_lengths = np.diff(run_starts)
        run_values = chunk[run_starts[:-1]]
        encoded_chunks.extend(zip(run_values, run_lengths))
    
    return encoded_chunks

def compress_large_binary_image(binary_image):
    """
    Compress a large 2D binary image using run-length encoding.
    """
    encoded_data = rle_encode_large(binary_image.ravel())
    return encoded_data, binary_image.shape

def rle_decode_large(encoded_data, shape):
    """
    Decode run-length encoded data back to a large 2D binary numpy array.
    This function decodes the data in chunks to save memory.
    """
    total_size = shape[0] * shape[1]
    decoded_array = np.zeros(total_size, dtype=np.uint8)
    current_index = 0

    for value, count in tqdm(encoded_data, desc="Decoding"):
        end_index = current_index + count
        decoded_array[current_index:end_index] = value
        current_index = end_index

    return decoded_array.reshape(shape)

In [ ]:
def generate_parasite_image(width, height, min_blob_size=0.25, max_blob_size=0.4):
    """
    Generate a simulated microscope image with a single binary blob representing a parasite.
    The blob is centered and its size is at least 25% of the smaller image dimension.
    """
    # Create a blank binary image
    image = np.zeros((height, width), dtype=np.uint8)
    
    # Calculate the center of the image
    center = (width // 2, height // 2)
    
    # Calculate the radius
    min_dimension = min(width, height)
    min_radius = int(min_blob_size * min_dimension // 2)
    max_radius = int(max_blob_size * min_dimension // 2)
    radius = np.random.randint(min_radius, max_radius)
    
    # Draw the circle
    cv2.circle(image, center, radius, 1, -1)
    
    return image

In [ ]:
def generate_veins(parasite_mask, coverage=0.1, num_iterations=5):
    """Generate vein-like structures within a parasite chunk."""
    veins = np.zeros_like(parasite_mask, dtype=np.uint8)
    
    # Calculate the target number of vein pixels
    target_vein_pixels = int(np.sum(parasite_mask) * coverage)
    
    # Start with random points within the parasite
    initial_points = np.random.rand(*parasite_mask.shape) < 0.01
    veins[initial_points & (parasite_mask == 1)] = 1
    
    kernel = np.ones((3, 3), np.uint8)
    for _ in range(num_iterations):
        veins = cv2.dilate(veins, kernel, iterations=1)
        veins = veins & parasite_mask
        new_points = (np.random.rand(*parasite_mask.shape) < 0.01) & (parasite_mask == 1)
        veins = veins | new_points
        
        if np.sum(veins) >= target_vein_pixels:
            break
    
    return veins

def simulate_dye_spread(veins, parasite_mask, num_iterations=10):
    """Simulate dye spreading from veins within the parasite chunk."""
    dye = veins.copy()
    kernel = np.ones((3, 3), np.uint8)
    
    for _ in range(num_iterations):
        dye = cv2.dilate(dye, kernel, iterations=1)
        dye = dye & parasite_mask
        dye = dye | veins
    
    return dye

def generate_dye_image(parasite_image, chunk_size=1000, coverage=0.1):
    """Generate a binary dye image for a single parasite blob using chunks."""
    height, width = parasite_image.shape
    dye_image = np.zeros((height, width), dtype=np.uint8)
    
    # Calculate total parasite area for coverage calculation
    total_parasite_area = np.sum(parasite_image)
    total_dye_pixels = int(total_parasite_area * coverage)
    current_dye_pixels = 0
    
    for y in tqdm(range(0, height, chunk_size), desc="Generating dye image"):
        for x in range(0, width, chunk_size):
            chunk = parasite_image[y:y+chunk_size, x:x+chunk_size]
            
            if np.sum(chunk) == 0:  # Skip empty chunks
                continue
            
            # Calculate remaining coverage for this chunk
            chunk_parasite_area = np.sum(chunk)
            chunk_coverage = min(1.0, max(0, (total_dye_pixels - current_dye_pixels) / chunk_parasite_area))
            
            veins = generate_veins(chunk, coverage=chunk_coverage)
            dye_chunk = simulate_dye_spread(veins, chunk)
            
            dye_image[y:y+chunk_size, x:x+chunk_size] = dye_chunk
            current_dye_pixels += np.sum(dye_chunk)
            
            if current_dye_pixels >= total_dye_pixels:
                break
        if current_dye_pixels >= total_dye_pixels:
            print("current_dye_pixels >= total_dye_pixels")
            break
    
    return dye_image


In [ ]:
# %pip install numpy opencv-python matplotlib
# Parameters
width, height = 100_000, 100_000
# Generate simulated images
print("Generating parasite image...")
parasite_image = generate_parasite_image(width, height)

In [ ]:
print("Compressing parasite image...")
parasite_rle, parasite_shape = compress_large_binary_image(parasite_image)
parasites_present = any(value == 1 for value, count in parasite_rle)
print("Parasites present in RLE:", parasites_present)
parasite_original_size = width * height // 8  # in bytes
parasite_compressed_size = sum(len(run) for run in parasite_rle) * 2  # 2 bytes per run (value and count)
parasite_compression_ratio = parasite_original_size / parasite_compressed_size
print(f"\nParasite Image - Original size: {parasite_original_size / 1e9:.2f} GB")
print(f"Parasite Image - Compressed size: {parasite_compressed_size / 1e9:.2f} GB")
print(f"Parasite Image - Compression ratio: {parasite_compression_ratio:.2f}x")

In [ ]:
print("Generating dye image...")
dye_image = generate_dye_image(parasite_image)


In [ ]:
print("Compressing dye image...")
dye_rle, dye_shape = compress_large_binary_image(dye_image)

In [ ]:
dye_original_size = width * height // 8  # in bytes
dye_compressed_size = sum(len(run) for run in dye_rle) * 2
dye_compression_ratio = dye_original_size / dye_compressed_size
print(f"\nDye Image - Original size: {dye_original_size / 1e9:.2f} GB")
print(f"Dye Image - Compressed size: {dye_compressed_size / 1e9:.2f} GB")
print(f"Dye Image - Compression ratio: {dye_compression_ratio:.2f}x")

### Question 3

In [12]:
def parasite_has_cancer(parasite_rle, dye_rle, chunk_size=10000, threshold=0.1):
    """
    Determine if a parasite has cancer based on dye coverage.
    
    Args:
    parasite_image (np.ndarray): Binary image of the parasite (1 for parasite, 0 for background)
    dye_image (np.ndarray): Binary image of the dye (1 for dye presence, 0 for absence)
    chunk_size (int): The size of the chunks to process the images
    threshold (float): The dye coverage threshold for cancer diagnosis (default: 0.1)
    
    Returns:
    bool: True if the parasite has cancer (dye coverage > threshold), False otherwise
    float: The actual dye coverage
    """
    
    parasite_image= rle_decode_large(parasite_rle, parasite_shape)
    dye_image = rle_decode_large(dye_rle, dye_shape)
    print("Decoding completed")
    # Ensure the images are binary and have the same shape
    assert parasite_image.shape == dye_image.shape, "Parasite and dye images must have the same shape"
    # assert set(np.unique(parasite_image)).issubset({0, 1}), "Parasite image must be binary"
    # assert set(np.unique(dye_image)).issubset({0, 1}), "Dye image must be binary"
    
    # Initialize the total areas
    total_parasite_area = 0
    total_dye_area_in_parasite = 0
    
    # Get the dimensions of the images
    height, width = parasite_image.shape
    
    # Calculate the total number of chunks to process for progress tracking
    total_chunks = (height // chunk_size + (height % chunk_size > 0)) * (width // chunk_size + (width % chunk_size > 0))
    
    # Process the images in chunks with progress information
    for y in tqdm(range(0, height, chunk_size), desc="Processing rows", total=total_chunks, unit="chunk"):
        for x in range(0, width, chunk_size):
            # Define the chunk boundaries
            y_end = min(y + chunk_size, height)
            x_end = min(x + chunk_size, width)
            
            # Extract the chunks
            parasite_chunk = parasite_image[y:y_end, x:x_end]
            dye_chunk = dye_image[y:y_end, x:x_end]
            
            # Calculate the areas in the chunk
            parasite_area = np.sum(parasite_chunk)
            dye_area_in_parasite = np.sum(dye_chunk & parasite_chunk)
            
            # Update the total areas
            total_parasite_area += parasite_area
            total_dye_area_in_parasite += dye_area_in_parasite
    
    # Ensure the parasite exists in the image
    if total_parasite_area == 0:
        raise ValueError("No parasite detected in the image")
    
    # Calculate the dye coverage
    dye_coverage = total_dye_area_in_parasite / total_parasite_area
    
    # Determine if the parasite has cancer
    has_cancer = dye_coverage > threshold
    
    return has_cancer, dye_coverage




has_cancer, dye_coverage = parasite_has_cancer(parasite_rle, dye_rle)

print(f"Parasite has cancer: {has_cancer}")
print(f"coverage: {dye_coverage}")


Decoding: 100%|██████████| 14620/14620 [00:01<00:00, 12676.26it/s]


Decoding completed


Processing rows:  10%|█         | 10/100 [00:09<01:22,  1.09chunk/s]

Parasite has cancer: True
coverage: 0.10033145524309504


### Quesiton 4

I am using a laptop, and I got kernel error when I tried to generate images of 100000*100000. I believe the memory runs out. So for question 3, I had to use the improved method of using trunks for processing image of 100k. So my answer for question 3 is the improved one. 

I am running this on my local computer. Normally, I would run it in cloud env such as google colab, which can utilize GPU for extrememly fast processing. 



### Question 5

I could not come up with effective compression for this question, so I used both claud ai and chatgpt for an answer

Based on my inqueries, Quadtree compression could be an answer. 

My understanding is that we represent the image in tree structure. Each node represent a group of bits with the value. We divide the square image into four quads, and keep diving until all bits in a quads are of the same value. Then such quad becomes a leaf. This could be an effective representation of the images besides the bitmap. 

But given the setting of this challenge, I belive quadtree does not necessarily produce a better run time/storage saving. 

I had experience using google colab before. When I trained machine learning model before and turned large neural network, I typically run the cloud using google colab and run on the GPU. When budget is sufficent, upgrading the hardware might be the most effective approach than dealing with compression techniques. 

### Question 6:

Tools I used for this challenge: 

Google(searching)
Chatgpt(explaining the data structure, such as bitmap)

I am good at writing prompts to the LLM, and ask for answers to my questions in my daily development task, and this can greatly increase my efficiency.

I used chatgpt for the API of the libraries, such as ny functions, and some nuances.

